Author: BinhNVH

In [33]:
import pandas as pd
import os

In [34]:
class_map = {
    "Aortic enlargement" : 0,
    "Atelectasis" : 1,
    "Calcification" : 2,
    "Cardiomegaly" : 3,
    "Consolidation" : 4,
    "ILD" : 5,
    "Infiltration" : 6,
    "Lung Opacity" : 7,
    "Nodule/Mass" : 8,
    "Other lesion" : 9,
    "Pleural effusion" : 10,
    "Pleural thickening" : 11,
    "Pneumothorax" : 12,
    "Pulmonary fibrosis" :13
}

In [35]:
def convert_to_yolo(bbox_file, output_file_yolo):
    bbox_df = pd.read_csv(bbox_file)
    # Load bounding box data and convert to YOLOv5 format
    bbox_df['x_min'] = bbox_df.apply(lambda row: (row.x_min)/row.width, axis =1)
    bbox_df['y_min'] = bbox_df.apply(lambda row: (row.y_min)/row.height, axis =1)
    bbox_df['x_max'] = bbox_df.apply(lambda row: (row.x_max)/row.width, axis =1)
    bbox_df['y_max'] = bbox_df.apply(lambda row: (row.y_max)/row.height, axis =1)
    bbox_df['x_mid'] = bbox_df.apply(lambda row: (row.x_max+row.x_min)/2, axis =1)
    bbox_df['y_mid'] = bbox_df.apply(lambda row: (row.y_max+row.y_min)/2, axis =1)
    bbox_df['w'] = bbox_df.apply(lambda row: (row.x_max-row.x_min), axis =1)
    bbox_df['h'] = bbox_df.apply(lambda row: (row.y_max-row.y_min), axis =1)
    bbox_df['area'] = bbox_df['w']*bbox_df['h']
    new_bbox_df = bbox_df[['class_id', 'x_mid', 'y_mid', 'w', 'h']]

    # Group bounding boxes by image ID and save as separate txt files
    image_ids = bbox_df["image_id"].unique()
    for image_id in image_ids:
        output_file = os.path.join(output_file_yolo, f"{image_id}.txt")
        image_bboxes = new_bbox_df[bbox_df["image_id"] == image_id]
        image_bboxes.to_csv(output_file, header=None, index=None, sep=" ")
    print("Conversion complete!")

In [37]:
os.makedirs(r'E:\XR-AI\datasets\vinbigdata\yolo_label',exist_ok = True)
input = r'E:\XR-AI\datasets\vinbigdata\train.csv'
output = r'E:\XR-AI\datasets\vinbigdata\yolo_label'
convert_to_yolo(input,output)